In [1]:
import h5py
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
from UoI_Lasso import UoI_Lasso
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

from pvc11_utils import PVC11
%matplotlib inline

In [2]:
pvc11 = PVC11(data_path='data/pvc11/spikes_gratings/data_monkey1_gratings.mat')

In [3]:
Y = pvc11.get_response_matrix(transform='square_root')

In [4]:
# create storage
coupling_coefs_lasso = np.zeros((pvc11.n_neurons, pvc11.n_neurons))
coupling_coefs_r2 = np.zeros((pvc11.n_neurons, pvc11.n_neurons))
coupling_coefs_bic = np.zeros((pvc11.n_neurons, pvc11.n_neurons))
coupling_coefs_aic = np.zeros((pvc11.n_neurons, pvc11.n_neurons))

intercepts_lasso = np.zeros(pvc11.n_neurons)
intercepts_r2 = np.zeros(pvc11.n_neurons)
intercepts_bic = np.zeros(pvc11.n_neurons)
intercepts_aic = np.zeros(pvc11.n_neurons)

# r2, bic, aic
scores_lasso = np.zeros((pvc11.n_neurons, 3))
scores_r2 = np.zeros((pvc11.n_neurons, 3))
scores_bic = np.zeros((pvc11.n_neurons, 3))
scores_aic = np.zeros((pvc11.n_neurons, 3))

In [5]:
for neuron in range(pvc11.n_neurons):
    print(neuron)
    
    # create design and response matrices
    X = np.delete(Y, neuron, axis=1)
    y = Y[:, neuron]
    
    # split up into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
    
    ### create fitting objects ###
    lasso = LassoCV(
        normalize=True,
        cv=10,
        max_iter=10000
    )
    
    uoi_r2 = UoI_Lasso(
        normalize=True,
        fit_intercept=True,
        n_boots_sel=50,
        n_boots_est=50,
        n_lambdas=50,
        estimation_score='r2',
        selection_thres_min=1,
        n_selection_thres=1
    )

    uoi_bic = UoI_Lasso(
        normalize=True,
        fit_intercept=True,
        n_boots_sel=50,
        n_boots_est=50,
        n_lambdas=50,
        estimation_score='BIC',
        selection_thres_min=1,
        n_selection_thres=1
    )

    uoi_aic = UoI_Lasso(
        normalize=True,
        fit_intercept=True,
        n_boots_sel=50,
        n_boots_est=50,
        n_lambdas=50,
        estimation_score='AIC',
        selection_thres_min=1,
        n_selection_thres=1
    )
    
    ### perform fits ###
    lasso.fit(
        X_train,
        y_train
    )

    uoi_r2.fit(
        X_train,
        y_train
    )

    uoi_bic.fit(
        X_train,
        y_train
    )

    uoi_aic.fit(
        X_train,
        y_train
    )
    
    ### store fits ###
    coupling_coefs_lasso[neuron, :] = np.insert(lasso.coef_, neuron, 0.)
    coupling_coefs_r2[neuron, :] = np.insert(uoi_r2.coef_, neuron, 0.)
    coupling_coefs_bic[neuron, :] = np.insert(uoi_bic.coef_, neuron, 0.)
    coupling_coefs_aic[neuron, :] = np.insert(uoi_aic.coef_, neuron, 0.)
    intercepts_lasso[neuron] = lasso.intercept_
    intercepts_r2[neuron] = uoi_r2.intercept_
    intercepts_bic[neuron] = uoi_bic.intercept_
    intercepts_aic[neuron] = uoi_aic.intercept_
    
    ### scoring ###
    y_pred_lasso = lasso.intercept_ + np.dot(X_test, lasso.coef_)
    scores_lasso[neuron, 0] = r2_score(y_test, y_pred_lasso)
    scores_lasso[neuron, 1] = pvc11.BIC(y_test, y_pred_lasso, np.count_nonzero(lasso.coef_) + 1)
    scores_lasso[neuron, 2] = pvc11.AIC(y_test, y_pred_lasso, np.count_nonzero(lasso.coef_) + 1)
    
    y_pred_r2 = uoi_r2.intercept_ + np.dot(X_test, uoi_r2.coef_)
    scores_r2[neuron, 0] = r2_score(y_test, y_pred_r2)
    scores_r2[neuron, 1] = pvc11.BIC(y_test, y_pred_r2, np.count_nonzero(uoi_r2.coef_) + 1)
    scores_r2[neuron, 2] = pvc11.AIC(y_test, y_pred_r2, np.count_nonzero(uoi_r2.coef_) + 1)

    y_pred_bic = uoi_bic.intercept_ + np.dot(X_test, uoi_bic.coef_)
    scores_bic[neuron, 0] = r2_score(y_test, y_pred_bic)
    scores_bic[neuron, 1] = pvc11.BIC(y_test, y_pred_bic, np.count_nonzero(uoi_bic.coef_) + 1)
    scores_bic[neuron, 2] = pvc11.AIC(y_test, y_pred_bic, np.count_nonzero(uoi_bic.coef_) + 1)

    y_pred_aic = uoi_aic.intercept_ + np.dot(X_test, uoi_aic.coef_)
    scores_aic[neuron, 0] = r2_score(y_test, y_pred_aic)
    scores_aic[neuron, 1] = pvc11.BIC(y_test, y_pred_aic, np.count_nonzero(uoi_aic.coef_) + 1)
    scores_aic[neuron, 2] = pvc11.AIC(y_test, y_pred_aic, np.count_nonzero(uoi_aic.coef_) + 1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105


In [22]:
print(np.mean(scores_lasso[:, 0]))
print(np.mean(scores_r2[:, 0]))
print(np.mean(scores_aic[:, 0]))
print(np.mean(scores_bic[:, 0]))

0.5536768624889712
0.5449561065691186
0.538213586866626
0.5178826778583644


In [21]:
print(np.mean(scores_r2[:, 2]))
print(np.mean(scores_lasso[:, 2]))
print(np.mean(scores_aic[:, 2]))
print(np.mean(scores_bic[:, 2]))

-350.1872260977994
-314.1975381870129
-385.58996221205933
-374.6830476097298


In [23]:
print(np.std(scores_aic[:, 0]))
print(np.std(scores_lasso[:, 0]))

0.197369973161849
0.19680160820668924


In [24]:
fits = h5py.File('pvc11_fits.h5', 'w')
fits['lasso/coupling_coefs'] = coupling_coefs_lasso
fits['lasso/intercepts'] = intercepts_lasso
fits['lasso/r2s'] = scores_lasso[:, 0]
fits['lasso/bics'] = scores_lasso[:, 1]
fits['lasso/aics'] = scores_lasso[:, 2]

fits['uoi_r2/coupling_coefs'] = coupling_coefs_r2
fits['uoi_r2/intercepts'] = intercepts_r2
fits['uoi_r2/r2s'] = scores_r2[:, 0]
fits['uoi_r2/bics'] = scores_r2[:, 1]
fits['uoi_r2/aics'] = scores_r2[:, 2]

fits['uoi_bic/coupling_coefs'] = coupling_coefs_bic
fits['uoi_bic/intercepts'] = intercepts_bic
fits['uoi_bic/r2s'] = scores_bic[:, 0]
fits['uoi_bic/bics'] = scores_bic[:, 1]
fits['uoi_bic/aics'] = scores_bic[:, 2]

fits['uoi_aic/coupling_coefs'] = coupling_coefs_aic
fits['uoi_aic/intercepts'] = intercepts_aic
fits['uoi_aic/r2s'] = scores_aic[:, 0]
fits['uoi_aic/bics'] = scores_aic[:, 1]
fits['uoi_aic/aics'] = scores_aic[:, 2]
fits.close()